# Application of Benford's Law in Image Forgey Detection

This notebook demonstrates the application of Benford's Law in detecting forged images.

Benford's Law states that in many naturally occurring collections of numbers, the distribution of the first digits follows the logarithmic distribution:

$$F_a = \log_{10}\frac{a + 1}{a}$$

where $a$ is the first digit and $F_a$ is the probability of the first digit being $a$.


In [ ]:
# Plot of Benford's Law distribution

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({"font.size": 11})

# Benford's Law distribution
DIGITS = np.arange(1, 10)
BENFORD = np.log10(1 + 1 / DIGITS)

# Plot
plt.margins(0, tight=True)
plt.plot(DIGITS, BENFORD, "s-", color="black", clip_on=False, markerfacecolor="none")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.title("Benford's Law Distribution")
plt.xlabel("First Digit")
plt.ylabel("Frequency")
plt.ylim(0.001, 0.35)
plt.tick_params(direction="in")

## Dataset

For this notebook, we will be using a dataset that closely matches the one stated in the paper "Benford's law applied to digital forensic analysis" by Fernandes et al (2023):

| Name                            |   Fake   |   Real   |
| ------------------------------- | :------: | :------: |
| Columbia Image Splicing Dataset |   180    |   180    |
| COVERAGE dataset                |   100    |   100    |
| CelebA-HQ dataset               |    -     |   8600   |
| This person does not exist      |   120    |    -     |
| 100K-Faces-HQ Datset            |   8600   |    -     |
| Flickr-Faces-HQ Dataset         |          |   120    |
|                                 |          |          |
| **Total**                       | **9000** | **9000** |

However, due to the paper's lack of clear details on the methodology (for example, the CelebA-HQ dataset consists of 30,000 images, but only 8,600 were used), the dataset recreated is done to the best of my abilities.


In [ ]:
import os

dir_fake = ["../dataset/fake"]
dir_real = ["../dataset/real"]


def gather_files(dirs: list[str]) -> list[str]:
    """Collect all file paths, filter invalid files and return a list of valid file paths."""
    return [
        os.path.join(subdir, file)
        for dir in dirs
        for subdir, _, files in os.walk(dir)
        for file in files
        if os.path.isfile(os.path.join(subdir, file))
    ]


fake_files = gather_files(dir_fake)
real_files = gather_files(dir_real)

print(f"Fake files: {len(fake_files)}, Real files: {len(real_files)}")

## Discrete Cosine Transform (DCT)

The methodology is as follows:

1. Read the image and convert it to grayscale to get its luminance channel.
2. Divide the image to non-overlapping 8x8 blocks.
3. Apply DCT to each block.
4. Extract the DCT coefficients and their first digits.
5. Accumulate the first digits of the DCT coefficients, and discard the 0 coefficients.
6. Use hypothesis testing to determine if the image is real or fake wrt Benford's Law.


In [ ]:
from functools import partial
from multiprocessing import Pool, cpu_count

import cv2
from tqdm import tqdm

import scipy as sp


def zigzag(matrix: np.ndarray) -> np.ndarray:
    """
    Zigzag traversal of a 2D matrix to convert a 2D array to a 1D array.
    """
    rows, cols = matrix.shape
    result = []
    for diag in range(rows + cols - 1):
        if diag % 2 == 0:
            # Even diagonal: traverse from bottom-left to top-right
            start_row = min(diag, rows - 1)
            start_col = diag - start_row
            while start_row >= 0 and start_col < cols:
                result.append(matrix[start_row, start_col])
                start_row -= 1
                start_col += 1
        else:
            # Odd diagonal: traverse from top-right to bottom-left
            start_col = min(diag, cols - 1)
            start_row = diag - start_col
            while start_col >= 0 and start_row < rows:
                result.append(matrix[start_row, start_col])
                start_row += 1
                start_col -= 1
    return np.array(result)


def dct(
    filename: str,
    block_size: int = 8,
) -> list[float]:
    """Perform FFT on an image and return the flattened DCT coefficients."""
    try:
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(f"Error reading {filename}: {e}")

    height, width = img.shape

    # Crop the image to a multiple of block_size
    height = height - height % block_size
    width = width - width % block_size

    img = img[:height, :width]

    blocks = img.reshape(
        height // block_size, block_size, width // block_size, block_size
    ).swapaxes(1, 2)

    dct_blocks = np.array(
        [
            [
                zigzag(sp.fft.dctn(blocks[i, j], norm="ortho"))
                for j in range(blocks.shape[1])
            ]
            for i in range(blocks.shape[0])
        ],
    )

    return dct_blocks.flatten()


def multithread_dct(filenames: list[str], **kwargs) -> list[list[float]]:
    with Pool(processes=cpu_count()) as pool:
        results = list(
            tqdm(
                pool.imap(partial(dct, **kwargs), filenames),
                total=len(filenames),
                desc="Performing Feature Extraction",
            )
        )
    return results


In [ ]:
dct_coeffs_fake = multithread_dct(fake_files)
dct_coeffs_real = multithread_dct(real_files)

# Remove None results if any files failed to process
dct_coeffs_fake = [result for result in dct_coeffs_fake if result is not None]
dct_coeffs_real = [result for result in dct_coeffs_real if result is not None]

In [ ]:
label_total_fake = np.full(len(dct_coeffs_fake), True)
label_total_real = np.full(len(dct_coeffs_real), False)

features = dct_coeffs_fake + dct_coeffs_real
labels = np.concatenate((label_total_fake, label_total_real))

## First Digits Extraction

To prepare our dataset for analysis, we will extract the first digits of the DFT coefficients of the images.


In [ ]:
def get_first_digit(value: float) -> int:
    """Get the first digit of a value."""
    value = abs(value)  # Ensure the number is positive
    while value >= 10:
        value //= 10  # Divide by 10 until it's less than 10
    while value < 1 and value > 0:
        value *= 10  # Multiply by 10 until it's at least 1
    return int(value)  # Return the integer part


def get_first_digit_array(array: list[float]) -> list[int]:
    """Get the first digit of each value in an array."""
    return [get_first_digit(value) for value in array]


def get_digit_counts(array: list[int]) -> list[int]:
    """Count the occurrences of each digit in an array."""
    return [array.count(digit) for digit in DIGITS]


with Pool(processes=cpu_count()) as pool:
    first_digits = list(
        tqdm(
            pool.imap(get_first_digit_array, features),
            total=len(features),
            desc="Getting First Digits",
        )
    )

    first_digits_counts = list(
        tqdm(
            pool.imap(get_digit_counts, first_digits),
            total=len(first_digits),
            desc="Counting First Digits",
        )
    )

## Hypothesis Testing

Here, we will do hypothesis testing to determine whether a picture is considered fake or not. We will use the Pearson's Correlation Coefficient of the distribution of the image's first digits count with respect to the Benford's Law.

If the absolute value of the Pearson's Correlation Coefficient $\rho$ is close to 1, then the first digit distribution follows Benford's Law, and we can consider the image as real. Otherwise, if the absolute value of the Pearson's Correlation Coefficient is not equal to 1, then the first digit distribution does not follow Benford's Law, and we can consider the image as fake.

$$H_0: \rho = 1 \text{(Real Image)}$$
$$H_1: \rho \neq 1 \text{(Fake Image)}$$


In [ ]:
import scipy.stats as stats
import sklearn.metrics as metrics


# H0: The first digits distribution follows Benford's Law
# H1: The first digits distribution does not follow Benford's Law

# Fail to reject H0 when absolute value of correlation coefficient is close to 1
# Reject H0 when absolute value of correlation coefficient is below 1 - alpha

# H0 is equal to false, H1 is equal to true
# So return true when absolute value of correlation coefficient is below 1 - alpha

# (I had to write the above to deconfuse myself)


def test_results(
    threshold: int,
    first_digits_counts: list[list[int]] = first_digits_counts,
) -> dict:
    """Test the goodness of fit for each feature and return the confusion matrix and performance metrics."""
    goodness_of_fit = [
        stats.pearsonr(first_digits_count, BENFORD)
        for first_digits_count in first_digits_counts
    ]

    # calculate True Positive, False Positive, True Negative, False Negative
    results = [abs(coeff) < threshold for coeff, _ in goodness_of_fit]

    # label for fake is 0/False, real is 1/True
    TN, FP, FN, TP = metrics.confusion_matrix(labels, results).ravel()

    return {
        "TN": TN,
        "FP": FP,
        "FN": FN,
        "TP": TP,
        "Precision": metrics.precision_score(labels, results),
        "Recall": metrics.recall_score(labels, results),
        "F1": metrics.f1_score(labels, results),
        "Accuracy": metrics.accuracy_score(labels, results),
    }

In [ ]:
# Print the table using pandas Dataframe
import pandas as pd

THRESHOLD = [0.999, 0.99, 0.95, 0.9, 0.8, 0.75]

with Pool(processes=cpu_count()) as pool:
    results = pool.map(test_results, THRESHOLD)

df = pd.DataFrame.from_records(results, index=THRESHOLD)
df.columns.name = "Threshold"
df